In [1]:
# !pip install numpy==1.19.5
os.environ["KERAS_BACKEND"] = "plaidml.bridge.keras"

import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense

2022-03-15 17:50:18.347721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 17:50:18.347744: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [120]:
def import_data():
    """
    Import data and split into features and labels
    """
    data = pd.read_csv('data_kia.csv').values
    x_data = data[:, :51]
    y_data = data[:, 52]
    return x_data, y_data

data, labels = import_data()
data

array([[268.8, 0.0, 5.2, ..., -8.8, 0, -3.4],
       [243.2, 0.0, 6.1, ..., -0.2, 0, -3.6],
       [217.6, 0.0, 5.2, ..., -0.2, 0, -3.6],
       ...,
       [345.6, 0.0, 6.6, ..., 0.0, 0, -13.2],
       [332.8, 0.0, 5.7, ..., 0.0, 0, -13.3],
       [281.6, 0.0, 5.2, ..., -0.1, 0, -13.3]], dtype=object)

In [129]:
def normalize_data(rows):
    max_columns = rows.max(axis=0)
    min_columns = rows.min(axis=0)
    norm_rows = np.empty(rows.shape)
    for i in range(rows.shape[1]):
        for j in range(rows.shape[0]):
            if ((max_columns[i]-min_columns[i])>0):
                norm_rows[j][i] = (rows[j][i] - min_columns[i])/(max_columns[i]-min_columns[i])
            else:
                norm_rows[j][i] = 0
    return norm_rows

normal_data = normalize_data(data)
normal_data


array([[0.02459016, 0.        , 0.052     , ..., 0.07650273, 0.        ,
        0.49030656],
       [0.02224824, 0.        , 0.061     , ..., 0.54644809, 0.        ,
        0.49011556],
       [0.01990632, 0.        , 0.052     , ..., 0.54644809, 0.        ,
        0.49011556],
       ...,
       [0.03161593, 0.        , 0.066     , ..., 0.55737705, 0.        ,
        0.48094738],
       [0.03044496, 0.        , 0.057     , ..., 0.55737705, 0.        ,
        0.48085188],
       [0.02576112, 0.        , 0.052     , ..., 0.55191257, 0.        ,
        0.48085188]])

In [121]:
def onehot_labels(labels, num_labels=10):
    """
    Converts label array to 1-hot encoded vectors
    """

    result = []

    for l in labels:
        label_vector = [0] * num_labels
        index = ord(l) - ord('A')
        label_vector[index] = 1

        result.append(label_vector)

    return np.array(result)

y_data = tf.convert_to_tensor(onehot_labels(labels), dtype=tf.float64)


y_data

<tf.Tensor: shape=(94380, 10), dtype=float64, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [ ]:
def add_noise(data, noise_level, std_level):
    induced_std = std_level * np.std(data, axis=1)
    noisy_data = np.empty((data.shape[]))
    for j in range(rows.shape[1]):
        for i in range(rows.shape[0]]):
            if (np.random.rand(1) >= noise_level):
                noisy_data[i][j] = data[i][j] + np.random.normal(0,induced_std[j])
            else: 
                nois
        
    

In [122]:
def package_data(data, window_size=16):
    result = []
    for i in range(len(data) - window_size):
        result.append(data[i:i + window_size])

    return result
x_data = tf.convert_to_tensor(package_data(data), dtype=tf.float64)
x_data[0].shape

2022-03-15 17:29:19.847886: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 616008192 exceeds 10% of free system memory.


TensorShape([16, 51])

In [126]:
def create_model(learning_rate=1e-4, window_size=16, num_features=51):
    model = Sequential()
    model.add(LSTM(160, input_shape=(window_size, num_features), return_sequences=True ))
    model.add(LSTM(200, ))

    model.add(Dense(10, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=['categorical_crossentropy'])

    return model

In [127]:
model = create_model()
model.fit(x_data, y_data[ :-16], epochs=10, batch_size=128)

738/738 [==============================] - 72s 94ms/step - loss: 2.0090 - categorical_crossentropy: 2.0090
